In [1]:
import pandas as pd
import os

In [3]:
# Mapeamento dos meses número -> nome

map_mes = {
    1.0: 'janeiro',
    2.0: 'fevereiro',
    3.0: 'março',
    4.0: 'abril',
    5.0: 'maio',
    6.0: 'junho',
    7.0: 'julho',
    8.0: 'agosto',
    9.0: 'setembro',
    10.0: 'outubro',
    11.0: 'novembro',
    12.0: 'dezembro'
}

In [21]:
path_ = '../input/dados_html/'

all_files = os.listdir(path_)

all_dfs = []

for file in all_files:
    if file.split(".")[-1] == 'html':
        df_ = pd.read_html(path_ + file, encoding='utf-8', thousands=".")[0]
        df_ = df_[['Ano','Mês','Amazônia','Caatinga','Cerrado','Mata Atlântica','Pampa','Pantanal','Total mensal']]
        df_ = df_[:-1]
        all_dfs.append(df_)

    
all_data = pd.concat(all_dfs, ignore_index=True)
all_data = all_data.sort_values(by=['Ano','Mês'])
all_data.reset_index(inplace=True, drop=True)
all_data['mes_nome'] = all_data['Mês'].map(map_mes)
all_data.rename(columns={"Ano": 'ano', 'Mês': 'mes', 'Amazônia': 'amazonia', 'Caatinga': 'caatinga', 'Cerrado': 'cerrado', 'Mata Atlântica': 'mata_atlantica', 'Pampa': 'pampa', 'Pantanal': 'pantanal', "Total mensal": 'total'}, inplace=True)


all_data

,ano,mes,amazonia,caatinga,cerrado,mata_atlantica,pampa,pantanal,total,mes_nome
0,2003,1.0,3528,2206,157,436,1.0,172.0,6500,janeiro
1,2003,2.0,1960,56,214,107,3.0,30.0,2370,fevereiro
2,2003,3.0,3295,80,258,286,1.0,38.0,3958,março
3,2003,4.0,1179,440,1322,421,13.0,56.0,3431,abril
4,2003,5.0,2033,248,5200,3321,316.0,179.0,11297,maio
...,...,...,...,...,...,...,...,...,...,...
223,2021,8.0,18560,3300,32070,4682,123.0,3196.0,61931,agosto
224,2021,9.0,10045,13604,57473,6775,9.0,8354.0,96260,setembro
225,2021,10.0,5840,12722,12799,1272,15.0,4931.0,37579,outubro
226,2021,11.0,2148,12151,438,272,30.0,318.0,15357,novembro


In [22]:
melt_data = all_data.melt(id_vars=['ano','mes','mes_nome'], value_vars=['amazonia','caatinga','cerrado','mata_atlantica','pantanal','pampa'])
melt_data.rename(columns={'variable': 'bioma', 'value': 'total'}, inplace=True)
melt_data

,ano,mes,mes_nome,bioma,total
0,2003,1.0,janeiro,amazonia,3528.0
1,2003,2.0,fevereiro,amazonia,1960.0
2,2003,3.0,março,amazonia,3295.0
3,2003,4.0,abril,amazonia,1179.0
4,2003,5.0,maio,amazonia,2033.0
...,...,...,...,...,...
1363,2021,8.0,agosto,pampa,123.0
1364,2021,9.0,setembro,pampa,9.0
1365,2021,10.0,outubro,pampa,15.0
1366,2021,11.0,novembro,pampa,30.0


In [25]:
melt_data['total_acumulado'] = melt_data.groupby(by=['ano','bioma'])['total'].cumsum()
melt_data

,ano,mes,mes_nome,bioma,total,total_acumulado
0,2003,1.0,janeiro,amazonia,3528.0,3528.0
1,2003,2.0,fevereiro,amazonia,1960.0,5488.0
2,2003,3.0,março,amazonia,3295.0,8783.0
3,2003,4.0,abril,amazonia,1179.0,9962.0
4,2003,5.0,maio,amazonia,2033.0,11995.0
...,...,...,...,...,...,...
1363,2021,8.0,agosto,pampa,123.0,1141.0
1364,2021,9.0,setembro,pampa,9.0,1150.0
1365,2021,10.0,outubro,pampa,15.0,1165.0
1366,2021,11.0,novembro,pampa,30.0,1195.0


In [60]:
melt_data.to_csv("../output/br_queimadas_area_bioma.csv",index=False)